In [2]:
import os
import ipyvolume as ipv
import tyssue
import json
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import random
import sys
import logging
import tyssue.io.hdf5 as hdf5
from IPython.display import Image
from scipy import optimize
from tyssue.draw import sheet_view

from tyssue import Sheet
from tyssue import SheetGeometry

##### Own functions
import src.cellDivisionOld as cellDivisionOld
import src.cellDivision as cellDivision
import src.vertexModel as vertexModel
import src.inputMechanicalParameters as inputMechanicalParameters
import src.auxFunctions as auxFunctions
from tyssue.topology import add_vert

from tyssue.draw.plt_draw import create_gif
from tyssue.draw import highlight_faces, create_gif
from IPython.display import Image

from tyssue.topology.base_topology import add_vert, close_face, collapse_edge, remove_face
from tyssue.topology.base_topology import split_vert as base_split_vert

from matplotlib import cm
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as mcolors

from math import sqrt
from scipy.optimize import curve_fit
from scipy.stats import ttest_ind
import json

import pickle 
import gzip

import logging
logger = logging.getLogger(name=__name__)

import warnings
warnings.filterwarnings("ignore")


collision solver could not be imported You may need to install CGAL and re-install tyssue


In [ ]:
#### Initialize Model
[cellmap_init, geom, energyContributions_model] = vertexModel.initialize()

## Update mechanical parameters
cellmap_init = inputMechanicalParameters.update(cellmap_init)

## Initial stage
energyContributions_model.compute_energy(cellmap_init)

## RUN
[cellmap_H, geom, model_H, history_H, solver1] = vertexModel.solveEuler(cellmap_init, geom, energyContributions_model, endTime = 100)


fig, ax = sheet_view(cellmap_H, ['y', 'x'], edge={"color":1, 'colormap': 'Greys'})
#auxFunctions.create_frames(history_H, './results', edge={'color':'black'})
#auxFunctions.exportToMesh(history_H, './results')


In [ ]:
#cellmap_H.remove(cellmap_H.cut_out([[5, 35], [5, 35]]), trim_borders=True)


fig, ax = sheet_view(cellmap_H)
fig.set_size_inches((15, 15))
plt.show()

In [ ]:
areas = cellmap_H.face_df['area']
area_positive = areas[areas >= 0]

plt.figure(figsize=(10, 6))
plt.hist(area_positive, color='skyblue', bins=100, edgecolor='black')
#plt.title('Histogram of Areas ')
plt.xlabel('Area')
plt.ylabel('Frequency')
plt.show()

In [ ]:
perimeter = cellmap_H.face_df['perimeter']

plt.figure(figsize=(10, 6))
plt.hist(perimeter, color='skyblue', bins=100, edgecolor='black')
plt.title('Histogram of T cell Perimeters')
plt.xlabel('Perimeter')
plt.ylabel('Frequency')
plt.show()

In [ ]:
lengths = cellmap_H.edge_df['length']
plt.figure(figsize=(10, 6))
plt.hist(lengths, color='skyblue', bins=100, edgecolor='black')
plt.title('Histogram of Cell Lengths')
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
from matplotlib import cm
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
def heatmap_of_edges(cellmap, parameter):

    # Dynamically retrieve the range of parameter values for proper scaling

    min_val = parameter.min()
    max_val = parameter.max()
    
    specs = {
        'face': {
            'visible': True,
            'color': parameter,  # This should be a column name in cellmap.edge_df
            'colormap': 'RdPu',
            'width': 2,
        },
        'edge': {
            'visible': False,
            
        },
        'vert': {
            'visible': False,
        }
    }

    fig, ax = sheet_view(cellmap, **specs)

    norm = plt.Normalize(vmin=min_val, vmax=max_val)
    sm = ScalarMappable(cmap=cm.RdPu, norm=norm)

    # Create an axes on the right side of ax. The width of cax will be 5%
    # of ax and the padding between cax and ax will be fixed at 0.05 inch.
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(sm, cax=cax)
    fig.set_size_inches(20, 20)
    plt.show()

In [ ]:
heatmap_of_edges(cellmap_H, cellmap_H.face_df['area'])

In [ ]:
num_sides_data = cellmap_H.face_df['num_sides']

# Generate bin edges such that each bar is centered on the integers
min_side = num_sides_data.min()
max_side = num_sides_data.max()
bin_edges = np.arange(min_side - 0.5, max_side + 1.5, 1)


# Create the histogram 

fig, ax = plt.subplots(figsize=(10, 6))

ax.set_title('Number of FRCs Enclosing T cell zones', fontsize=18, fontweight='bold')
ax.set_xlabel('Number of Enclosing FRC branches', fontsize=14, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=14, fontweight='bold')

# Customize the figure and axes background color
fig.patch.set_facecolor('white')
ax.set_facecolor('white')


ax.hist(num_sides_data, bins=bin_edges, color='skyblue', alpha=0.9, rwidth=0.85, edgecolor='skyblue')



In [ ]:
FRC_ECM_ratios = []

for index, row in cellmap_H.edge_df.iterrows():
    length_elasticity_FRC = row['length_elasticity_FRC']
    length_elasticity_ECM = row['length_elasticity_ECM']
    ratio = length_elasticity_FRC / length_elasticity_ECM
    FRC_ECM_ratios.append(ratio)
    
min(FRC_ECM_ratios)

In [ ]:
FRC_ECM_ratios = []

for index, row in cellmap_H.edge_df.iterrows():
    length_elasticity_FRC = row['length_elasticity_FRC']
    length_elasticity_ECM = row['length_elasticity_ECM']
    ratio = length_elasticity_FRC / length_elasticity_ECM
    # Append a tuple of (ratio, edge ID)
    FRC_ECM_ratios.append((ratio, index))
    
# Find the tuple with the minimum ratio
min_ratio_tuple = min(FRC_ECM_ratios, key=lambda x: x[0])

# Extract the ratio and edge ID from the tuple
min_ratio, edge_id_with_min_ratio = min_ratio_tuple

print("Minimum ratio:", min_ratio)
print("Edge ID with minimum ratio:", edge_id_with_min_ratio)

edge_id_with_min_ratio

In [ ]:
cellmap_H, new_edges_indices, new_opp_edges_indices, new_vertices_indices = cellDivision.ln_divide_FRCs(cellmap_H, "length", 0.3, energyContributions_model)

In [ ]:
import seaborn as sns
area = cellmap_H.face_df['area']
area_positive = area[area >= 0]
sns.violinplot(y = area_positive, orient='v')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

    
cellmap_H.edge_df.loc[ten_longest_edges, "color_edges"] = 256
cellmap_H.edge_df.loc[ten_longest_edges, 'edge_width'] = 4




specs = {
    'face': {
        'visible': False,
    },
    'edge': {
            'visible': True,
            'color': cellmap_H.edge_df['color_edges'],
            'colormap': 'coolwarm',
            'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}



fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

ax.set_facecolor('#EFEFEF')  # Set background color to light gray


# Add grid lines
ax.grid(True, linestyle='--', linewidth=0.5, color='gray')

#ax.set_zorder(0)  # Set a lower zorder value to move the background to the back

plt.title('Spatial distribution of longest FRCs', fontsize = 20)

plt.show()

In [ ]:
### match length_elasticity of opposite edges 

for edge in range(len(cellmap_H.edge_df)):
    
   ### find the opposite edge of each edge in edges_list_without_opposites
    
    opposite_edge_indices = cellmap_H.edge_df[
        (cellmap_H.edge_df["srce"] == cellmap_H.edge_df.loc[edge, "trgt"]) &
        (cellmap_H.edge_df["trgt"] == cellmap_H.edge_df.loc[edge, "srce"])
    ].index
    


    if len(opposite_edge_indices):
            # Update the values for the opposite edges
            cellmap_H.edge_df['length_elasticity_ECM'][opposite_edge_indices] = cellmap_H.edge_df['length_elasticity_ECM'][edge] 
            cellmap_H.edge_df['length_elasticity_FRC'][opposite_edge_indices] = cellmap_H.edge_df['length_elasticity_FRC'][edge] 
            cellmap_H.edge_df['length_elasticity'][opposite_edge_indices] = cellmap_H.edge_df['length_elasticity'][edge]

In [ ]:
### match prefered_length of opposite edges 


for edge in range(len(cellmap_H.edge_df)):
    
    ### find the opposite edge of each edge in edges_list_without_opposites
    
    opposite_edge_indices = cellmap_H.edge_df[
        (cellmap_H.edge_df["srce"] == cellmap_H.edge_df.loc[edge, "trgt"]) &
        (cellmap_H.edge_df["trgt"] == cellmap_H.edge_df.loc[edge, "srce"])
    ].index


    if len(opposite_edge_indices):
            # Update the values for the opposite edges
            cellmap_H.edge_df['prefered_length'][opposite_edge_indices] = cellmap_H.edge_df['prefered_length'][edge] 
            

In [ ]:
cellmap_H.edge_df

In [ ]:
def drop_edges(cellmap, condition, number):
    
    # Find non-border edges
    edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap)
    
    # Choose edges to remove from non-border edges
    edges_to_remove = cellmap.edge_df[~cellmap.edge_df.index.isin(border_edges_indices)][condition].nlargest(number*2).index.tolist()
    print(len(edges_to_remove))
    
    cellmap_H.edge_df["color_edges"] = 0
    cellmap_H.edge_df['edge_width'] = 1

    cellmap_H.edge_df.loc[edges_to_remove, "color_edges"] = 1
    cellmap_H.edge_df.loc[edges_to_remove, 'edge_width'] = 4

    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': True,
            'color': cellmap.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'color' : '#000a4b',
            's' : 20,
        }
    }



    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))


    
    # Remove edges_to_remove from edge_df
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(edges_to_remove)]
    
    # Remove any unused vertices 
    # cellmap.vert_df = cellmap.vert_df.loc[remaining_verts]
    geom.update_all(cellmap)
    cellmap.reset_index()
    cellmap.reset_topo()
    
    
    # Plot the graph after apoptosis
    fig, ax = sheet_view(cellmap_H)
    fig.set_size_inches((15, 15))


In [ ]:
drop_edges(cellmap_H, "length", 5)

In [ ]:
edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap_H)

num_rows = len(cellmap_H.edge_df)
num_sample = int(num_rows * 0.05)

# Choose random indices
random_edge_idx = np.random.choice(cellmap_H.edge_df.loc[edges_without_opposites_indices].index, size=num_sample, replace=False)
    
cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

cellmap_H.edge_df.loc[1, "color_edges"] = 1
cellmap_H.edge_df.loc[1, 'edge_width'] = 4

specs = {
    'face': {
        'visible': False,
    },
    'edge': {
            'visible': True,
            'color': cellmap_H.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}



fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

In [ ]:
associated_faces = cellmap_H.edge_df.loc[random_edge_idx]['face'].tolist()
cellmap_H.face_df.loc[associated_faces]['area']

In [ ]:
cellmap_H.face_df["color_faces"] = 0


cellmap_H.face_df.loc[associated_faces, "color_faces"] = 1

specs = {
    'face': {
        'visible': True,
        'color': cellmap_H.face_df["color_faces"],
        'colormap':'bwr',
    },
    'edge': {
        'visible': False,
       
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}

fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

In [ ]:
drop_edges(cellmap_H, "length", 10)

In [ ]:
sorted_face_df = cellmap_H.face_df.sort_values(by='area', ascending=False)
#sorted_face_df.head(10)['area'].tolist()
top_10 = sorted_face_df.head(10).index.tolist()

In [ ]:
energyContributions_model.compute_energy(cellmap_H)
[cellmap_H, geom, model_H, history_H] = vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, endTime = 100)

In [ ]:
fig, ax = sheet_view(cellmap_H)
fig.set_size_inches((15, 15))

In [ ]:
associated_faces

In [ ]:
cellmap_H.face_df["color_faces"] = 0


cellmap_H.face_df.loc[associated_faces, "color_faces"] = 1

specs = {
    'face': {
        'visible': True,
        'color': cellmap_H.face_df["color_faces"],
        'colormap':'bwr',
    },
    'edge': {
        'visible': False,
       
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}

fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

In [ ]:
def drop_one_edge(cellmap, condition):
    
    # Find non-border edges
    edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap)
    
    # Choose edge to remove from non-border edges
    edge_to_remove = cellmap.edge_df[~cellmap.edge_df.index.isin(border_edges_indices)][condition].nlargest(1).index.tolist()
    #edge_to_remove = np.random.choice(cellmap_H.edge_df.index)
    
    srce, trgt = cellmap.edge_df.loc[edge_to_remove[0], ["srce", "trgt"]]
    opposites = cellmap.edge_df[
    (cellmap.edge_df["srce"] == trgt) & (cellmap.edge_df["trgt"] == srce)]
    opp_edge = opposites.index
    
    
    cellmap.edge_df["color_edges"] = 0
    cellmap.edge_df['edge_width'] = 1

    cellmap.edge_df.loc[edge_to_remove, "color_edges"] = 1
    cellmap.edge_df.loc[edge_to_remove, 'edge_width'] = 4

    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': True,
            'color': cellmap.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'color' : '#000a4b',
            's' : 20,
        }
    }



    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))

    
    # Remove the edge_to_remove from edge_df
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(edge_to_remove)]
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(opp_edge)]


    fig, ax = sheet_view(cellmap_H)
    fig.set_size_inches((15, 15))

       
    # Remove any unused vertices 
    #cellmap.vert_df = cellmap.vert_df.loc[remaining_verts]
    
    # Reset index and topology (adjust as needed)
    cellmap.reset_index()
    cellmap.reset_topo()
    
    # Update geometry
    geom.update_all(cellmap)
    
    return edge_to_remove
    

In [ ]:
cellmap_H.face_df

In [ ]:
drop_one_edge(cellmap_H, "length")

In [ ]:
num_of_loops = int(len(cellmap_H.edge_df)*0.05)

loops = set()


for i in range(1, num_of_loops + 1): 
    
    edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap_H)

    random_edge_idx = np.random.choice(edges_without_opposites_indices)

    srce, trgt = cellmap_H.edge_df.loc[random_edge_idx, ["srce", "trgt"]]
    opposites = cellmap_H.edge_df[(cellmap_H.edge_df["srce"] == trgt) & (cellmap_H.edge_df["trgt"] == srce)]
    opp_edge = opposites.index
    
    
    cellmap_H.edge_df = cellmap_H.edge_df[~cellmap_H.edge_df.index.isin([random_edge_idx])]
    cellmap_H.edge_df = cellmap_H.edge_df[~cellmap_H.edge_df.index.isin(opp_edge.tolist())]  
    
    cellmap_H.reset_index()
    cellmap_H.reset_topo()
    
    geom.update_all(cellmap_H)
    
    energyContributions_model.compute_energy(cellmap_H)
    vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)
    
    loops.add(i)
    
    print(len(loops))
    
    if len(loops) == num_of_loops:
        break
        
    
fig, ax = sheet_view(cellmap_H, mode="quick")
fig.set_size_inches(20, 20)
plt.show() 

In [ ]:
drop_one_edge(cellmap_H, "length")
    
geom.update_all(cellmap_H)
    
energyContributions_model.compute_energy(cellmap_H)
vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)

In [ ]:
### remove a % of edges by dropping minimising energy after each drop 

edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap_H)

# Calculate the number of edges to select
number = int(len(edges_without_opposites_indices)*0.15)

print(number)

# Iterate until the desired number of edges have been selected
selected_edges = set()

while len(selected_edges) < 20:
    
    # Get an edge index to remove
    edge_to_remove = drop_one_edge(cellmap_H, "length")
    print('edge removed')
    selected_edges.add(edge_to_remove.index)
    
    #minimize energxy
    energyContributions_model.compute_energy(cellmap_H)
    [cellmap_H, geom, model_H, history_H] = vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, endTime = 100)
    
    print(len(selected_edges))
    if len(selected_edges) == num_to_select:
        break



In [ ]:
fig, ax = sheet_view(cellmap_H)
fig.set_size_inches((15, 15))

In [ ]:
edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap_H)



In [ ]:
edge_to_remove_index = random.choice(edges_without_opposites_indices)  # Use random.choice on the list of indices

edge_to_remove_index

In [ ]:
cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

cellmap_H.edge_df.loc[edges_without_opposites_indices, "color_edges"] = 1
cellmap_H.edge_df.loc[edges_without_opposites_indices, 'edge_width'] = 4

specs = {
    'face': {
        'visible': False,
    },
    'edge': {
        'visible': True,
        'color': cellmap_H.edge_df["color_edges"],
        'colormap':'bwr',
        'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}
    
fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))
    
plt.show()

In [ ]:
def apoptosis_drop_one_FRC(cellmap, condition):
    
    # Find non-border edges
    edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap)
    
    # Choose edge to remove from non-border edges
    #edge_to_remove = cellmap.edge_df[~cellmap.edge_df.index.isin(border_edges_indices)][condition].nsmallest(1).index.tolist()
    
    edge_to_remove = random.choice(edges_without_opposites_indices)
  
    
    srce, trgt = cellmap.edge_df.loc[edge_to_remove, ["srce", "trgt"]]
    opposites = cellmap.edge_df[
    (cellmap.edge_df["srce"] == trgt) & (cellmap.edge_df["trgt"] == srce)]
    opp_edge = opposites.index
    
    cellmap.edge_df["color_edges"] = 0
    cellmap.edge_df['edge_width'] = 1

    cellmap.edge_df.loc[edge_to_remove, "color_edges"] = 1
    cellmap.edge_df.loc[edge_to_remove, 'edge_width'] = 4

    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': True,
            'color': cellmap.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'color' : '#000a4b',
            's' : 20,
        }
    }
    
    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))
    
    plt.show()
    
    # Remove the edge_to_remove from edge_df
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(edge_to_remove)]
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(opp_edge)]


    # Reset index and topology (adjust as needed)
    cellmap.reset_index()
    cellmap.reset_topo()
    
    # Update geometry
    geom.update_all(cellmap)
    
    energyContributions_model.compute_energy(cellmap)
    vertexModel.solveEuler(cellmap, geom, energyContributions_model, 40)
    
    fig, ax = sheet_view(cellmap_H)
    fig.set_size_inches((15, 15))

    
    return edge_to_remove
    

In [ ]:
def apoptosis_drop_one_FRC(cellmap, condition):
    
    # Find non-border edges
    edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap)
    
    # Choose random edge to remove from non-border edges
    
    edge_to_remove = random.choice(edges_without_opposites_indices)
  
    
    srce, trgt = cellmap.edge_df.loc[edge_to_remove, ["srce", "trgt"]]
    opposites = cellmap.edge_df[
    (cellmap.edge_df["srce"] == trgt) & (cellmap.edge_df["trgt"] == srce)]
    opp_edge = opposites.index
    
    edge_to_remove = [edge_to_remove]
    
    cellmap.edge_df["color_edges"] = 0
    cellmap.edge_df['edge_width'] = 1

    cellmap.edge_df.loc[edge_to_remove, "color_edges"] = 1
    cellmap.edge_df.loc[edge_to_remove, 'edge_width'] = 4

    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': True,
            'color': cellmap.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'color' : '#000a4b',
            's' : 20,
        }
    }
    
    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))
    
    plt.show()
    
    # Remove the edge_to_remove from edge_df
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(edge_to_remove)]
    cellmap.edge_df = cellmap.edge_df[~cellmap.edge_df.index.isin(opp_edge)]


    # Reset index and topology (adjust as needed)
    cellmap.reset_index()
    cellmap.reset_topo()
    
    # Update geometry
    geom.update_all(cellmap)
    
    energyContributions_model.compute_energy(cellmap)
    vertexModel.solveEuler(cellmap, geom, energyContributions_model, 40)
    
    fig, ax = sheet_view(cellmap_H)
    fig.set_size_inches((15, 15))

    
    return edge_to_remove
    

In [ ]:
cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

cellmap_H.edge_df.loc[1, "color_edges"] = 1
cellmap_H.edge_df.loc[1, 'edge_width'] = 4

specs = {
    'face': {
        'visible': False,
    },
    'edge': {
        'visible': True,
        'color': cellmap_H.edge_df["color_edges"],
        'colormap':'bwr',
        'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}
    
fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))
    
plt.show()

In [ ]:
apoptosis_drop_one_FRC(cellmap_H, "length")

In [ ]:
#num_of_loops = int(len(cellmap_H.edge_df/2*(15/100)))
num_of_loops = 50

loops = set()


for i in range(1, num_of_loops + 1): 
    cellDivision.ln_divide_one_FRC(cellmap_H, "length", energyContributions_model, geom)
    apoptosis_drop_one_FRC(cellmap_H, "length")

    loops.add(i)
    print(len(loops))
    if len(loops) == num_of_loops:
        break

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(area_positive, color='skyblue', bins=100, edgecolor='black')
#plt.title('Histogram of Areas ')
plt.xlabel('Area')
plt.ylabel('Frequency')
plt.show()

In [ ]:
perimeter = cellmap_H.face_df['perimeter']

plt.figure(figsize=(10, 6))
plt.hist(perimeter, color='skyblue', bins=100, edgecolor='black')
plt.title('Histogram of T cell Perimeters')
plt.xlabel('Perimeter')
plt.ylabel('Frequency')
plt.show()

In [ ]:
lengths = cellmap_H.edge_df['length']
plt.figure(figsize=(10, 6))
plt.hist(lengths, color='skyblue', bins=100, edgecolor='black')
plt.title('Histogram of Cell Lengths')
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
from matplotlib import cm
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
def heatmap_of_edges(cellmap, parameter):

    # Dynamically retrieve the range of parameter values for proper scaling

    min_val = parameter.min()
    max_val = parameter.max()
    
    specs = {
        'face': {
            'visible': True,
            'color': parameter,  # This should be a column name in cellmap.edge_df
            'colormap': 'RdPu',
            'width': 2,
        },
        'edge': {
            'visible': False,
            
        },
        'vert': {
            'visible': False,
        }
    }

    fig, ax = sheet_view(cellmap, **specs)

    norm = plt.Normalize(vmin=min_val, vmax=max_val)
    sm = ScalarMappable(cmap=cm.RdPu, norm=norm)

    # Create an axes on the right side of ax. The width of cax will be 5%
    # of ax and the padding between cax and ax will be fixed at 0.05 inch.
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(sm, cax=cax)
    fig.set_size_inches(20, 20)
    plt.show()

In [ ]:
heatmap_of_edges(cellmap_H, cellmap_H.face_df['area'])

In [ ]:
num_sides_data = cellmap_H.face_df['num_sides']

# Generate bin edges such that each bar is centered on the integers
min_side = num_sides_data.min()
max_side = num_sides_data.max()
bin_edges = np.arange(min_side - 0.5, max_side + 1.5, 1)


# Create the histogram 

fig, ax = plt.subplots(figsize=(10, 6))

ax.set_title('Number of FRCs Enclosing T cell zones', fontsize=18, fontweight='bold')
ax.set_xlabel('Number of Enclosing FRC branches', fontsize=14, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=14, fontweight='bold')

# Customize the figure and axes background color
fig.patch.set_facecolor('white')
ax.set_facecolor('white')


ax.hist(num_sides_data, bins=bin_edges, color='skyblue', alpha=0.9, rwidth=0.85, edgecolor='skyblue')




In [ ]:
cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

#cellmap_H.edge_df.loc[10, "color_edges"] = 1
#cellmap_H.edge_df.loc[10, 'edge_width'] = 4

specs = {
    'face': {
        'visible': False,
    },
    'edge': {
            'visible': True,
            'color': cellmap_H.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}



fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))



### Apoptosis by merging vertices

### Choosing edge by low ECM 

In [ ]:
edge_id = 0  # Replace with the actual edge ID you're interested in

# Access the row for the specified edge ID
edge_row = cellmap_H.edge_df.loc[edge_id]

# Extract the source and target vertex IDs from the row
source_vertex_id = edge_row['srce']
target_vertex_id = edge_row['trgt']

print("Source vertex ID for edge {}: {}".format(edge_id, source_vertex_id))
print("Target vertex ID for edge {}: {}".format(edge_id, target_vertex_id))

In [ ]:
    cellmap_H.edge_df["color_edges"] = 0
    cellmap_H.edge_df['edge_width'] = 1

    cellmap_H.edge_df.loc[0, "color_edges"] = 1
    cellmap_H.edge_df.loc[0, 'edge_width'] = 4
    
    
    cellmap_H.vert_df["color_vert"] = 0
    cellmap_H.vert_df.loc[6, "color_vert"] = 1


    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': False,
            'color': 'none',  # Assuming you intend to use 'color_edges' for a direct color specification, you might need to adjust this.
            'colormap': 'bwr',  # This may not be effective if 'color' is used directly.
            'width': cellmap_H.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'c': cellmap_H.vert_df["color_vert"],  # This should be used for numerical data to be mapped with 'cmap'
            'cmap': 'bwr',
            's': 20,
        }
    }

    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))
    

In [ ]:
def merge_vertices(sheet, vert0, vert1, reindex=True):
    """Merge the two vertices vert0 and vert1 if they are linked by an edge

    If `reindex` is `True` (the default), resets indexes and topology data

    Note:
    -----
    It is more efficient to call directly `collapse_edge`

    """
    logger.debug(f"merging vertices {vert0, vert1}")

    edges = sheet.edge_df[
        ((sheet.edge_df["srce"] == vert0) & (sheet.edge_df["trgt"] == vert1))
        | ((sheet.edge_df["srce"] == vert1) & (sheet.edge_df["trgt"] == vert0))
    ].index
    if not len(edges):
        raise ValueError(
            f"""No edge found between vertices {vert0} and {vert1}, cannot merge"""
        )
    return collapse_edge(sheet, edges[0], reindex)

import logging
logger = logging.getLogger(name=__name__)


In [ ]:
#num_of_loops = int(len(cellmap_H.edge_df)*0.005)
num_of_loops = 1

FRC_ECM_ratios = []


loops = set()

for i in range(1, num_of_loops + 1): 
    
    for index, row in cellmap_H.edge_df.iterrows():
        length_elasticity_FRC = row['length_elasticity_FRC']
        length_elasticity_ECM = row['length_elasticity_ECM']
    
        ratio = length_elasticity_FRC / length_elasticity_ECM
    
        # Append a tuple of (ratio, edge ID)
    
        FRC_ECM_ratios.append((ratio, index))
    
    
        # Find the tuple with the minimum ratio
        min_ratio_tuple = min(FRC_ECM_ratios, key=lambda x: x[0])
    
        # Extract the ratio and edge ID from the tuple
        min_ratio, edge_id_with_min_ratio = min_ratio_tuple

    
    edge_to_remove = edge_id_with_min_ratio
    
    print(edge_to_remove)
    

    # Access the row for the specified edge ID
    edge_row = cellmap_H.edge_df.loc[edge_to_remove]

    # Extract the source and target vertex IDs from the row
    source_vertex_id = edge_row['srce']
    target_vertex_id = edge_row['trgt']

    print("Source vertex ID for edge {}: {}".format(edge_to_remove, source_vertex_id))
    print("Target vertex ID for edge {}: {}".format(edge_to_remove, target_vertex_id))
    
    cellmap_H.edge_df["color_edges"] = 0
    cellmap_H.edge_df['edge_width'] = 1

    cellmap_H.edge_df.loc[edge_to_remove, "color_edges"] = 1
    cellmap_H.edge_df.loc[edge_to_remove, 'edge_width'] = 4

    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': True,
            'color': cellmap_H.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap_H.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'color' : '#000a4b',
            's' : 20,
        }
    }



    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))
    
    # Use edge_idx to collapse the edge, perform updates, and computations
    merge_vertices(cellmap_H, source_vertex_id, target_vertex_id, reindex=True)  
    cellmap_H.update_rank()
    

    geom.update_all(cellmap_H)
    
    print(len(cellmap_H.edge_df))
    
    fig, ax = sheet_view(cellmap_H, mode="quick")
    fig.set_size_inches(20, 20)
    plt.show() 
    
    energyContributions_model.compute_energy(cellmap_H)
    [cellmap_H, geom, model_H, history_H, solver1] = vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)
    
    
    fig, ax = sheet_view(cellmap_H, mode="quick")
    fig.set_size_inches(20, 20)
    plt.show() 
    
    loops.add(i)
    
    print(len(loops))
    
    if len(loops) == num_of_loops:
        break

fig, ax = sheet_view(cellmap_H, mode="quick")
fig.set_size_inches(20, 20)
plt.show() 





In [ ]:
#num_of_loops = int(len(cellmap_H.edge_df)*0.005)
num_of_loops = 50


loops = set()

for i in range(1, num_of_loops + 1): 
    
    edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap_H)
    
    # find edge with lowest ECM 

    edge_to_remove = cellmap_H.edge_df['length_elasticity_ECM'].nsmallest(1).index.tolist()

    #edge_to_remove = np.random.choice(cellmap_H.edge_df.index)
    
    print(edge_to_remove)
    
    cellmap_H.edge_df["color_edges"] = 0
    cellmap_H.edge_df['edge_width'] = 1

    cellmap_H.edge_df.loc[edge_to_remove, "color_edges"] = 1
    cellmap_H.edge_df.loc[edge_to_remove, 'edge_width'] = 4

    specs = {
        'face': {
            'visible': False,
        },
        'edge': {
            'visible': True,
            'color': cellmap_H.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap_H.edge_df['edge_width'],
        },
        'vert': {
            'visible': True,
            'color' : '#000a4b',
            's' : 20,
        }
    }



    fig, ax = sheet_view(cellmap_H, **specs)
    fig.set_size_inches((15, 15))
    
    # Use edge_idx to collapse the edge, perform updates, and computations
    collapse_edge(cellmap_H, edge_to_remove[0])  
    cellmap_H.update_rank()
    
    fig, ax = sheet_view(cellmap_H, mode="quick")
    fig.set_size_inches(20, 20)
    plt.show() 

    geom.update_all(cellmap_H)
    energyContributions_model.compute_energy(cellmap_H)
    [cellmap_H, geom, model_H, history_H, solver1] = vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)
    
    loops.add(i)
    
    print(len(loops))
    
    if len(loops) == num_of_loops:
        break

fig, ax = sheet_view(cellmap_H, mode="quick")
fig.set_size_inches(20, 20)
plt.show() 

In [ ]:
def view(cellmap):
    #cellmap.remove(cellmap.cut_out([[5, 35], [5, 35]]), trim_borders=True)
    geom.update_all(cellmap)
    if cellmap.edge_df['length_elasticity'].max():
        ecolor = cellmap.edge_df['length_elasticity']
    else:
        ecolor = "#aaaaaa"
        
    fig, ax = sheet_view(
        cellmap,
        mode="2D",
        face={"visible": False},
        edge={"head_width": 0.0, "color": ecolor, "width": 2},
        vert={"visible": False}
    )
    fig.set_size_inches(20, 20)
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.set_xlim([0, 500])  # X-axis shows data from 0 to 100
    ax.set_ylim([0, 500]) 
    
    return fig, ax

In [ ]:
from tyssue.draw.plt_draw import create_gif
from tyssue.draw import highlight_faces, create_gif
from IPython.display import Image

In [ ]:
create_gif(solver1.history, "apoptosis.gif", draw_func=view, num_frames=101)

In [ ]:
Image("apoptosis.gif")

In [ ]:
num_rows = len(cellmap_H.edge_df)
num_sample = int(num_rows * 0.05)

# Choose random indices
random_edge_idx = np.random.choice(cellmap_H.edge_df.index, size=num_sample, replace=False)
    
cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

cellmap_H.edge_df.loc[random_edge_idx, "color_edges"] = 1
cellmap_H.edge_df.loc[random_edge_idx, 'edge_width'] = 4

specs = {
    'face': {
        'visible': False,
    },
    'edge': {
            'visible': True,
            'color': cellmap_H.edge_df["color_edges"],
            'colormap':'bwr',
            'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}



fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

In [ ]:
random_edge_idx

In [ ]:
for edge_idx in random_edge_idx:
    # Use edge_idx to collapse the edge, perform updates, and computations
    collapse_edge(cellmap_H, edge_idx)  
    print(edge_idx)  # This now prints the actual index of the edge being processed
    cellmap_H.update_rank()

    geom.update_all(cellmap_H)
    energyContributions_model.compute_energy(cellmap_H)
    vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)

fig, ax = sheet_view(cellmap_H, mode="quick")
fig.set_size_inches(20, 20)
plt.show() 

In [ ]:
merge_vertices(cellmap_H, 5, 6, reindex=True)

In [ ]:
len(cellmap_H.edge_df)

In [ ]:
edges_to_consider = cellmap_H.edge_df[~cellmap_H.edge_df.index.isin(border_edges_indices)]
largest_edge = edges_to_consider["length"].nlargest(1).index[0]  # Access the first item directly
largest_edge
collapse_edge(cellmap_H, largest_edge, allow_two_sided=False)


In [ ]:
edges_to_consider = cellmap_H.edge_df[~cellmap_H.edge_df.index.isin(border_edges_indices)]

if not edges_to_consider.empty:
    largest_edge = edges_to_consider["length"].nlargest(1).index[0]  # Access the first item directly
    collapse_edge(cellmap_H, largest_edge, allow_two_sided=False)
else:
    print("No suitable edge found for collapsing.")

In [ ]:
def drop_two_sided_faces(eptm):
    """Removes all the two (or one?) sided faces from the epithelium

    Note that they are not collapsed, but simply eliminated
    Does not reindex
    """

    num_sides = eptm.edge_df.groupby("face").size()
    if num_sides.min() > 2:
        return

    two_sided = eptm.face_df[num_sides < 3].index
    logger.debug("dropping %d 2-sided faces", two_sided.size)
    edges = eptm.edge_df[eptm.edge_df["face"].isin(two_sided)].index
    eptm.edge_df.drop(edges, axis=0, inplace=True)
    eptm.face_df.drop(two_sided, axis=0, inplace=True)

In [ ]:
len(cellmap_H.edge_df)

In [ ]:
cellDivision.ln_divide_one_FRC(cellmap_H, "length", energyContributions_model)



In [ ]:
num_of_loops = 50

loops = set()

for i in range(1, num_of_loops + 1): 


    random_edge_idx = np.random.choice(cellmap_H.edge_df.index)

    collapse_edge(cellmap_H, random_edge_idx)
    
    
    geom.update_all(cellmap_H)
    
    energyContributions_model.compute_energy(cellmap_H)
    vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)
    
    print("FRC killed")
    
    print(len(cellmap_H.vert_df))
    
    cellDivision.ln_divide_one_FRC(cellmap_H, "length", energyContributions_model, geom)
    print("FRC divided")
    print(len(cellmap_H.edge_df))
    

    loops.add(i)
    print(len(loops))
    if len(loops) == num_of_loops:
        break

In [ ]:
cellmap_H.edge_df

In [ ]:
collapse_edge(cellmap_H, 8114, reindex=True, allow_two_sided=False)

In [ ]:
fig, ax = sheet_view(cellmap_H)
fig.set_size_inches((15, 15))

In [ ]:
len(cellmap_H.edge_df)

In [ ]:
edges_without_opposites_indices, border_edges_indices = cellDivision.edges_list_without_opposites(cellmap_H)
edge_to_remove = cellmap_H.edge_df.loc[edges_without_opposites_indices]["length"].idxmax()

cellmap_H.edge_df["color_edges"] = 0
cellmap_H.edge_df['edge_width'] = 1

cellmap_H.edge_df.loc[edge_to_remove, "color_edges"] = 1
cellmap_H.edge_df.loc[edge_to_remove, 'edge_width'] = 4

cellmap_H.face_df["color_faces"] = 0
cellmap_H.face_df.loc[associated_face, "color_faces"] = 1

specs = {
    'face': {
        'visible': True,
        'color': cellmap_H.face_df["color_faces"],
        'colormap':'bwr',
    },
    'edge': {
        'visible': True,
        'color': cellmap_H.edge_df["color_edges"],
        'colormap':'bwr',
        'width' : cellmap_H.edge_df['edge_width'],
    },
    'vert': {
        'visible': True,
        'color' : '#000a4b',
        's' : 20,
    }
}
    
fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

In [ ]:
associated_face = cellmap_H.edge_df.loc[edge_to_remove]['face']


In [ ]:
collapse_edge(cellmap_H, edge_to_remove, allow_two_sided=False)
cellmap_H.update_rank()
    
geom.update_all(cellmap_H)
    
energyContributions_model.compute_energy(cellmap_H)
vertexModel.solveEuler(cellmap_H, geom, energyContributions_model, 40)
    
fig, ax = sheet_view(cellmap_H, **specs)
fig.set_size_inches((15, 15))

### Apoptosis by only one edge contraction

In [ ]:
def view(cellmap, geom):
    """
    Visualizes the cell map with options to show vertices and adjust vertex size.
    """
    # Update the geometry
    geom.update_all(cellmap)

    # Set edge color to a solid black
    ecolor = "blue"  # Solid color for edges
    
    # Create the plot
    fig, ax = sheet_view(
        cellmap,
        mode="2D",
        face={"visible": False},
        edge={"head_width": 0.0, "color": ecolor, "width": 2, "alpha": 1.0},  # Solid edges, full opacity
        vert={"visible": True, "s": 50, "color": "red", "alpha": 1.0}  # Use 's' for vertex size, "color" for vertex color
    )
    fig.set_size_inches(20, 20)
    ax.set_xticks([])
    ax.set_yticks([])

    return fig, ax


In [ ]:
def save_gif_and_frames(frame_list, output_gif_path, duration=0.5):
    """
    Helper function to save all frames as a GIF.
    Parameters:
    - frame_list: List of paths to the image files that will be included in the GIF.
    - output_gif_path: The output file path for the GIF.
    - duration: The duration (in seconds) for each frame in the GIF.
    """
    with imageio.get_writer(output_gif_path, mode='I', duration=duration) as writer:
        for frame_path in frame_list:
            image = imageio.imread(frame_path)
            writer.append_data(image)
    print(f"GIF created and saved as {output_gif_path}")


In [ ]:
import os
import numpy as np
import imageio
import matplotlib.pyplot as plt

def merge_random_edge_and_save_frames(sheet, geom, energyContributions_model, frame_folder, frame_count, all_frames):
    """
    Merges the two vertices connected by a randomly selected edge into a single new vertex,
    reconnects the edges to this new vertex, and saves frames before and after each step.
    Also solves for energy after each merge.
    """
    # Ensure the frame_folder exists
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)
    
    # Step 1: Choose a random edge
    chosen_edge = sheet.edge_df.sample().index[0]
    logger.info(f"Chosen edge: {chosen_edge}")

    # Identify the vertices connected by the chosen edge
    edge = sheet.edge_df.loc[chosen_edge]
    srce_vertex = edge['srce']
    trgt_vertex = edge['trgt']

    logger.info(f"Vertices to be merged: {srce_vertex}, {trgt_vertex}")
    
    # Step 2: Save frame before deletion
    fig, ax = auxFunctions.view(sheet, geom)  # Assuming you have a `view` function to visualize the sheet
    frame_filename = os.path.join(frame_folder, f"frame_{frame_count:03d}_before.png")
    fig.savefig(frame_filename, dpi=60)
    plt.close(fig)
    all_frames.append(frame_filename)
    frame_count += 1
    
    # Step 3: Merge the two vertices by redirecting the edges
    new_vertex_id = max(sheet.vert_df.index) + 1
    new_vertex_data = {
        'x': (sheet.vert_df.loc[srce_vertex, 'x'] + sheet.vert_df.loc[trgt_vertex, 'x']) / 2,
        'y': (sheet.vert_df.loc[srce_vertex, 'y'] + sheet.vert_df.loc[trgt_vertex, 'y']) / 2
    }
    sheet.vert_df.loc[new_vertex_id] = new_vertex_data
    
    connected_edges = sheet.edge_df[
        (sheet.edge_df["srce"] == srce_vertex) | (sheet.edge_df["trgt"] == srce_vertex) |
        (sheet.edge_df["srce"] == trgt_vertex) | (sheet.edge_df["trgt"] == trgt_vertex)
    ]
    
    for _, edge in connected_edges.iterrows():
        if edge['srce'] == srce_vertex or edge['srce'] == trgt_vertex:
            sheet.edge_df.at[edge.name, 'srce'] = new_vertex_id
        if edge['trgt'] == srce_vertex or edge['trgt'] == trgt_vertex:
            sheet.edge_df.at[edge.name, 'trgt'] = new_vertex_id

    # Remove old vertices
    sheet.vert_df.drop([srce_vertex, trgt_vertex], inplace=True)
    
    # Remove the old edge
    sheet.edge_df.drop(chosen_edge, inplace=True)
    
    logger.info(f"Merged vertices {srce_vertex} and {trgt_vertex} into vertex {new_vertex_id}.")

    # Step 4: Update geometry and topology
    sheet.reset_index()
    sheet.reset_topo()
    geom.update_all(sheet)

    # **New Step**: Solve energy after merging and save frames from solver history
    energyContributions_model.compute_energy(sheet)
    [sheet, geom, model_H, history_H, solver1] = vertexModel.solveEuler(sheet, geom, energyContributions_model, endTime=10)

    logger.info(f"Energy recomputed after merging vertices into vertex {new_vertex_id}.")

    # Save frames for each step in the solver history
    for i, (t, cellmap_during_step) in enumerate(solver1.history):
        fig, ax = auxFunctions.view(cellmap_during_step, geom)  # Visualize the sheet
        frame_filename = os.path.join(frame_folder, f"frame_{frame_count:03d}_step.png")
        fig.savefig(frame_filename, dpi=60)
        plt.close(fig)

        all_frames.append(frame_filename)
        frame_count += 1
    
    logger.info(f"Frames saved after merging vertices into vertex {new_vertex_id}.")

    return frame_count

# Example usage
frame_folder = 'test test'

# Ensure the folder for all PNG frames exists before starting
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)

frame_count = 0  # Initialize frame_count
all_frames = []  # Initialize the list to store frame paths

# Initialize the cellmap (this should be done once before the loop)
print("Saving frames for the initial cellmap...")
[cellmap_init, geom, energyContributions_model] = vertexModel.initialize()
cellmap_init = inputMechanicalParameters.update(cellmap_init)
energyContributions_model.compute_energy(cellmap_init)
[cellmap_init, geom, model_init, history_init, solver1_init] = vertexModel.solveEuler(cellmap_init, geom, energyContributions_model, endTime=100)

# Save initial frames
for i, (t, cellmap_during_step) in enumerate(solver1_init.history):
    fig, ax = auxFunctions.view(cellmap_during_step, geom)
    frame_filename = os.path.join(frame_folder, f"initial_frame_{frame_count:03d}.png")
    fig.savefig(frame_filename, dpi=60)
    plt.close(fig)
    all_frames.append(frame_filename)
    frame_count += 1

# Loop to collapse 50 random edges in batches of 10 and save frames
for batch in range(5):  # Change from 10 to 5 batches
    for i in range(10):
        logger.info(f"Collapsing edge {batch * 10 + i + 1}/50")
        frame_count = merge_random_edge_and_save_frames(cellmap_init, geom, energyContributions_model, frame_folder, frame_count, all_frames)
        logger.info(f"Energy recomputed after collapsing edge {batch * 10 + i + 1}/50")
    
    # Optionally, you could perform additional operations after each batch here
    logger.info(f"Batch {batch + 1}/5 completed.")  # Adjusted to 5 batches

# Combine all PNGs into one final GIF
gif_path = os.path.join(frame_folder, 'test test.gif')  # Adjusted to 50 edges in the filename
save_gif_and_frames(all_frames, gif_path)

print(f"GIF created at: {gif_path}")


In [ ]:
dynamic_divide_and_apoptosis_random_adjacent_collapse(cellmap_init, gif_folder='illustration', max_cycles=1, max_layers=2)
